In [ ]:
import paramiko
from os.path import join

In [ ]:
paramiko.util.log_to_file("paramiko.log")

In [ ]:
host = "10.29.216.206"
port = 22
username = "boletim"
password = "#sesa#etl"

command = "cd gal; ./python3 exporta-complemento.py"

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

stdin, stdout, stderr = ssh.exec_command(command)
lines = stdout.readlines()
print("".join(lines))

path_file = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'gal')
remotePath = "/home/boletim/gal/consulta-gal-complemento.csv"
localPath = join(path_file,'consulta-gal-complemento.csv')

## Copy remote file to server        
sftp = ssh.open_sftp()
sftp.get(remotePath,localPath)

##
sftp.close()
ssh.close()

In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, chdir, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from datetime import date, timedelta, datetime
from os.path import dirname, join, isfile, isdir
from bulletin import default_input, default_output, hoje
from bulletin.utils import utils, static, normalize

path_file = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'gal')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [ ]:

utils.create_backup(first_name = "backup_notifica_diario_" , level=3)
chdir(current_dir)

In [ ]:
from bulletin.utils.clean_up import clear_directories
clear_directories()

In [ ]:
today = date.today()
someday = date(2020, 1, 2)
diff = today - someday

In [ ]:
# COLUNAS DO BANCO DO GAL

# Index(['id'0, 'origem_gal'1, 'requisicao'2, 'data_requisicao'3, 'data_solicitacao'4,
#         'requisicao_correlativo'5, 'unidade_requisitante'6,
#         'unidade_requisitante_municipio'7, 'unidade_requisitante_uf'8, 'paciente'9,
#         'cns'10, 'cpf'11, 'nome_mae'12, 'sexo'13, 'data_nascimento'14, 'idade'15,
#         'raca_cor'16, 'idade_gestacional'17, 'ibge'18, 'municipio'19, 'latitude'20,
#         'longitude'21, 'codigo_regional'22, 'regional'23, 'data_inicio_sintomas'24,
#         'cod_exame_requisicao'25, 'data_coleta'26, 'codigo_amostra'27,
#         'numero_amostra'28, 'numero_interno'29, 'material_biologico'30,
#         'data_encaminhamento_amostra'31, 'data_recebimento_amostra'32,
#         'tempo_transporte_amostra'33, 'envio_oportuno'34, 'exame'35, 'metodo'36,
#         'resultado'37, 'virus'38, 'data_liberacao'39, 'laboratorio_executor'40,
#         'laboratorio_executor_municipio'41, 'correlativo_exame'42,
#         'tempo_liberacao'43, 'tempo_diagnostico'44, 'codigo_status'45, 'status'46,
#         'classificacao'47],
#        dtype='object')]

In [ ]:
gal_2020 = pd.read_pickle(join(path_file, 'banco_gal_2020-bz2.pkl'), compression='bz2')
gal_2020 = gal_2020[['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]
# gal_2020[['id', 'data_requisicao']].groupby('data_requisicao').count().sort_values('data_requisicao')

In [ ]:
gal_2021 = pd.read_pickle(join(path_file, 'banco_gal_2021-bz2.pkl'), compression='bz2')
gal_2021 = gal_2021[['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]
gal_2021[['id', 'data_requisicao']].groupby('data_requisicao').count().sort_values('data_requisicao')

In [ ]:
gal_complemento = pd.read_csv(join(path_file, 'consulta-gal-complemento.csv'), sep = ';', usecols=['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao'])
gal_complemento[['id', 'data_requisicao']].groupby('data_requisicao').count().sort_values('data_requisicao')

In [ ]:
# gal = pd.concat([gal_2020, gal_2021])
gal = pd.concat([gal_2020, gal_2021, gal_complemento])
# gal[['id', 'data_requisicao']].groupby('data_requisicao').count().sort_values('data_requisicao')

In [ ]:
boletim = gal.loc[gal["virus"] == 'Coronavírus SARS-CoV2']
# boletim = boletim.loc[boletim['unidade_requisitante_uf'] == 'PARANA']

In [ ]:
notificados = boletim[['id', 'status']].groupby('status').count().reset_index()

In [ ]:
descartado = boletim.loc[boletim['classificacao'] == 'Descartado']
descartado = descartado[['id', 'ibge', 'municipio', 'regional']].groupby(['ibge', 'municipio', 'regional']).count().reset_index()
descartado = descartado.rename(columns={'id':'Descartado'})
# descartado

In [ ]:
confirmado = boletim.loc[boletim['classificacao'] == 'Confirmado']
confirmado = confirmado[['id', 'ibge', 'municipio', 'regional']].groupby(['ibge', 'municipio', 'regional']).count().reset_index()
confirmado = confirmado.rename(columns={'id':'Confirmado'})
# confirmado

In [ ]:
# em_analise = gal.loc[gal['unidade_requisitante_uf'] == 'PARANA']
em_analise = gal.loc[gal['classificacao'] == 'Em investigação']
em_analise = em_analise[['id', 'ibge', 'municipio','regional']].groupby(['ibge','municipio', 'regional']).count().reset_index()
em_analise = em_analise.rename(columns={'id':'Em investigação'})

In [ ]:
# em_triagem = gal.loc[gal['unidade_requisitante_uf'] == 'PARANA']
em_triagem = gal.loc[gal['classificacao'] == 'Em triagem']
em_triagem = em_triagem[['id', 'ibge', 'municipio','regional']].groupby(['ibge', 'municipio', 'regional']).count().reset_index()
em_triagem = em_triagem.rename(columns={'id':'Em triagem'})

In [ ]:
nao_realizado = gal.loc[gal['classificacao'] == 'Não realizado']
nao_realizado = nao_realizado[['id', 'ibge', 'municipio','regional']].groupby(['ibge', 'municipio', 'regional']).count().reset_index()
nao_realizado = nao_realizado.rename(columns={'id':'Não realizado'})

In [ ]:
tabela = pd.DataFrame()
tabela = pd.merge(left=descartado, right=em_analise, how='left', on=['ibge', 'municipio', 'regional'])
tabela = pd.merge(left=tabela, right=confirmado, how='left', on=['ibge', 'municipio', 'regional'])
tabela = pd.merge(left=tabela, right=em_triagem, how='left', on=['ibge', 'municipio', 'regional'])
tabela = pd.merge(left=tabela, right=nao_realizado, how='left', on=['ibge', 'municipio', 'regional'])

In [ ]:
tabela['ibge'] = tabela['ibge'].fillna(0).astype('int')
tabela['Descartado'] = tabela['Descartado'].fillna(0).astype('int')
tabela['Em investigação'] = tabela['Em investigação'].fillna(0).astype('int')
tabela['Confirmado'] = tabela['Confirmado'].fillna(0).astype('int')
tabela['Em triagem'] = tabela['Em triagem'].fillna(0).astype('int')
tabela['Não realizado'] = tabela['Não realizado'].fillna(0).astype('int')

tabela.loc[tabela['regional'] == 'Outra UF', 'ibge'] = 999999

tabela = tabela.rename(columns={'ibge':'IBGE', 'municipio':'Municipio', 'regional':'Regional'})
tabela

In [ ]:
tabela.to_csv(join(default_output, 'enviar_celepar', 'CORONAVIRUS.analise.csv'), index=False, sep=';', encoding='utf-8-sig')

In [ ]:
adicionar_gal = tabela.rename(columns={'Em investigação':'Em_investigacao','Em triagem':'Em_triagem','Não realizado':'Nao_realizado'})
adicionar_gal = adicionar_gal.sort_values('Descartado', ascending=False)
adicionar_gal.to_csv(join(default_input,'Adicionar_GAL.csv'), index=False, sep=';', encoding='utf-8-sig')

In [ ]:
laboratorios = boletim[['id', 'laboratorio_executor']].groupby('laboratorio_executor').count().reset_index()
laboratorios = laboratorios.sort_values('id', ascending=False)
laboratorios = laboratorios.rename(columns={'id':'qtde'})
laboratorios.to_csv(join(default_output, 'dash', 'Laboratorios.csv'), index=False, sep=';', encoding='utf-8')

In [ ]:
soma = pd.DataFrame(tabela.sum(0)).transpose()
print('Quantidade de dias que DEVERIA TER = ', diff.days-12)
print('Quantidade de dias ENCONTRADOS = ', len(gal[['id', 'data_requisicao']].groupby('data_requisicao').count()))
print('\n')
print('NOTIFICADOS = ',  re.sub(r'(?<!^)(?=(\d{3})+$)', r'.', str(notificados['id'].values[0])))
print('REALIZADOS = ', re.sub(r'(?<!^)(?=(\d{3})+$)', r'.',str(tabela[['Descartado', 'Em investigação', 'Confirmado']].sum(0).sum(0))))
print('CONFIRMADO = ',  re.sub(r'(?<!^)(?=(\d{3})+$)', r'.', str(soma['Confirmado'].values[0])))
print('NEGATIVOS = ',  re.sub(r'(?<!^)(?=(\d{3})+$)', r'.',str(soma['Descartado'].values[0])))
print('EM INVESTIGAÇÃO = ',  re.sub(r'(?<!^)(?=(\d{3})+$)', r'.',str(soma['Em investigação'].values[0])))

In [ ]:
dados_gal = pd.DataFrame([notificados['id'].values[0], tabela[['Descartado', 'Em investigação', 'Confirmado']].sum(0).sum(0), soma['Confirmado'].values[0], soma['Descartado'].values[0], soma['Em investigação'].values[0]], columns=['count'])
# testes.to_csv(join(default_output,'enviar_celepar','CORONAVIRUS.testes.csv'), index=False, sep=';', encoding='utf-8-sig')
dados_gal.to_csv(join(default_input,'dados_gal.csv'), index=False, sep=';', encoding='utf-8')

In [ ]:
# gal['data_requisicao'] = pd.to_datetime(gal['data_requisicao'].str[:10])
# gal_mes = gal.groupby([gal['data_requisicao'].rename('ano').dt.year,gal['data_requisicao'].rename('mes').dt.month,'ibge'])[['id']].count()
# gal_mes['rank'] = gal_mes['id'].rank(ascending=False)
# gal_mes = gal_mes.rename(index={i+1:n for i,n in enumerate(static.meses)}, level=1)
# gal_mes['rank'] = gal_mes['rank'].astype(int)
# gal_mes = gal_mes[['id','rank']].rename(columns={'id':'quantidade'})
# gal_mes.reset_index().to_excel('rank_quantidade_rt_pcr_mes.xlsx', index=False)

In [ ]:
# gal['data_requisicao'] = pd.to_datetime(gal['data_requisicao'].str[:10])

In [ ]:
# gal = gal.loc[gal['ibge'].apply(str).str[:2]=='41']
# gal['ibge'] = gal['ibge'].apply(str).str[:6].apply(int)

In [ ]:
# gal_por_mun = gal.loc[gal['data_requisicao'] >= (hoje - timedelta(7))].groupby(['ibge'])[['id']].count()

In [ ]:
# gal['ibge'] = gal['ibge'].apply(str).str[:6].apply(int)

In [ ]:
# gal_por_mun['rank'] = gal_por_mun['id'].rank(ascending=False)

In [ ]:
# from bulletin.systems import CasosConfirmados
# cc = CasosConfirmados()
# municipios = cc.tables['municipios'].set_index('ibge')
# gal_por_mun = gal_por_mun.sort_values('rank').join(municipios[['rs','municipio']])
# gal_por_mun

In [ ]:
# gal_por_mun = gal_por_mun[['rs','municipio','id']].rename(columns={'id':'qtde'})
# gal_por_mun

In [ ]:
# gal_por_mun.to_excel('gal_por_mun_06ao13_01_2022.xlsx')